In [1]:
# conda install -c conda-forge gmaps
# ! pip install --user gmaps

In [2]:
# import packages
import gmaps
import pandas as pd
from api_keys import g_key
import requests
import json
from pprint import pprint

In [3]:
# api_key
gmaps.configure(api_key=g_key)

In [4]:
# read data
data = pd.read_csv("retrieved_data.csv")

# store lat long in variable
coordinates = data[["lat","long"]]

# humidity
humidity = data["humidity"].astype(float)
data.head()

,lat,long,closest city,country code,temp,humidity,cloudiness,wind speed
0,8.918051,84.955594,batticaloa,lk,76.08,70,22,4.72
1,-39.339486,26.073336,port elizabeth,za,58.10,91,31,7.34
2,-76.087878,157.534549,bluff,nz,66.79,48,0,7.83
3,-4.894565,5.401483,omboue,ga,77.49,90,96,9.37
4,-81.266040,25.806062,bredasdorp,za,60.12,82,100,4.61


In [5]:
# map
figure = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity,
                                 dissipating=False, max_intensity=1,
                                 point_radius= 1)

# Add layer
figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Ideal weather conditions

In [6]:
# filter for ideal weather conditions
data = data[data["temp"]  >= 68]
data = data[data["temp"]  <= 77]

data =data[data["cloudiness"]<50]
data

,lat,long,closest city,country code,temp,humidity,cloudiness,wind speed
0,8.918051,84.955594,batticaloa,lk,76.08,70,22,4.72
6,22.930503,-80.805127,marti,cu,70.07,76,9,10.31
11,21.293007,59.186946,sur,om,71.62,54,0,3.22
16,-37.856279,-45.595117,rio grande,br,68.95,80,0,13.35
20,-46.303532,78.319107,mahebourg,mu,75.49,88,40,3.44
...,...,...,...,...,...,...,...,...
534,2.957155,19.343059,gemena,cd,70.41,95,46,2.55
537,-31.394527,-50.622850,tapes,br,69.04,77,6,12.39
582,-22.218708,-73.868589,tocopilla,cl,69.98,75,3,5.03
585,16.169922,-161.738059,nanakuli,us,73.09,65,1,14.99


In [7]:
# geocoordinates
coordinates = "-31.9523, 115.8613"
search = "lodging"
radius = 5000

# set up a parameters dictionary
params = {
    "location": coordinates,
    "types": search,
    "radius": radius,
    "key": g_key
}

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()

In [15]:
data['Hotel Name'] = " "
search = search
radius = 5000
params = {
    "types": "hotel",
    "radius": radius,
    "key": g_key
    }

# find hotel names
print("Starting search...")
for index, row in data.iterrows():
        
    coordinates = f"{row['lat']},{row['long']}"
    params["location"] = coordinates
    try:
        response = requests.get(base_url, params=params).json()
        data.loc[index,'Hotel Name']  =response['results'][0]['name']
    except (KeyError, IndexError):
        data.loc[index,'Hotel Name']  = "Nan"
        print('Cant find hotel... skipping')
print('Search Done')

Starting search...
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
Cant find hotel... skipping
C

In [13]:
# hotels
hotels = data
data
# data["Hotel Name"].unique()

,lat,long,closest city,country code,temp,humidity,cloudiness,wind speed,Hotel Name
0,8.918051,84.955594,batticaloa,lk,76.08,70,22,4.72,Nan
6,22.930503,-80.805127,marti,cu,70.07,76,9,10.31,Nan
11,21.293007,59.186946,sur,om,71.62,54,0,3.22,Nan
16,-37.856279,-45.595117,rio grande,br,68.95,80,0,13.35,Nan
20,-46.303532,78.319107,mahebourg,mu,75.49,88,40,3.44,Nan
...,...,...,...,...,...,...,...,...,...
534,2.957155,19.343059,gemena,cd,70.41,95,46,2.55,Nan
537,-31.394527,-50.622850,tapes,br,69.04,77,6,12.39,Nan
582,-22.218708,-73.868589,tocopilla,cl,69.98,75,3,5.03,Nan
585,16.169922,-161.738059,nanakuli,us,73.09,65,1,14.99,Nan


In [11]:
# marker co ordinates
mark_cord = hotels[['lat', 'long']]


markers = gmaps.marker_layer(mark_cord)
figure.add_layer(markers)
figure

Figure(layout=FigureLayout(height='420px'))